In [7]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [16]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [17]:
df = pd.read_sql_query("Select * from FACILITIES", engine)

In [18]:
query = """
SELECT 
  sub2.name AS facilityname, 
  sub2.totalrevenue AS totalrevenue 
FROM 
  (
    SELECT 
      sub1.facilityname AS name, 
      SUM(sub1.revenue) AS totalrevenue 
    FROM 
      (
        SELECT 
          b.bookid, 
          f.name AS facilityname, 
          CASE WHEN b.memid = 0 THEN (b.slots * f.guestcost) ELSE b.slots * f.membercost END AS Revenue 
        FROM 
          Bookings AS b 
          LEFT JOIN Members AS m ON m.memid = b.memid 
          LEFT JOIN Facilities AS f ON f.facid = b.facid
      ) AS sub1 
    GROUP BY 
      sub1.facilityname
  ) AS sub2 
GROUP BY 
  facilityname 
HAVING 
  totalrevenue < 1000 
ORDER BY 
  totalrevenue DESC;
"""
pd.read_sql_query(query, engine)

,facilityname,totalrevenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


In [15]:
db_file

NameError: name 'db_file' is not defined

In [19]:
#PART 2: SQLite

#Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
#for the following questions. 

#QUESTIONS:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! */

query = """
SELECT sub2.name AS facilityname, sub2.totalrevenue AS totalrevenue 
FROM 
  (SELECT sub1.facilityname AS name, SUM(sub1.revenue) AS totalrevenue 
    FROM 
      (SELECT b.bookid, f.name AS facilityname, CASE WHEN b.memid = 0 THEN (b.slots * f.guestcost) ELSE b.slots * f.membercost END AS Revenue 
        FROM Bookings AS b 
          LEFT JOIN Members AS m ON m.memid = b.memid 
          LEFT JOIN Facilities AS f ON f.facid = b.facid) AS sub1 
    GROUP BY sub1.facilityname) AS sub2 
GROUP BY facilityname 
HAVING totalrevenue < 1000 
ORDER BY totalrevenue DESC;
"""
pd.read_sql_query(query, engine)

,facilityname,totalrevenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


In [20]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

query = """
select sub2.membername as membername, sub2.recommenderfirstname || ', ' || sub2.recommendersurname as recommendername
from
(select sub1.membername as membername, sub1.recommenderid as memberid, m.firstname as recommenderfirstname, m.surname as recommendersurname
from
(select m2.memid as memberid, m1.firstname || ', ' || m1.surname as membername, m2.recommendedby as recommenderid
from Members m1
inner join Members m2 on m1.memid = m2.memid
where
(m2.recommendedby is not null
or m2.recommendedby <> ' '
or m2.recommendedby <> '')
and m1.memid <> 0) as sub1
left join Members m on sub1.recommenderid = m.memid
where m.memid <> 0) as sub2;
"""
pd.read_sql_query(query, engine)

,membername,recommendername
0,"Janice, Joplette","Darren, Smith"
1,"Gerald, Butters","Darren, Smith"
2,"Nancy, Dare","Janice, Joplette"
3,"Tim, Boothe","Tim, Rownam"
4,"Ponder, Stibbons","Burton, Tracy"
5,"Charles, Owen","Darren, Smith"
6,"David, Jones","Janice, Joplette"
7,"Anne, Baker","Ponder, Stibbons"
8,"Jack, Smith","Darren, Smith"
9,"Florence, Bader","Ponder, Stibbons"


In [21]:

#Q12: Find the facilities with their usage by member, but not guests */

query = """
select f.name as facilityname, sum(b.slots) as slot_usage
from Bookings as b
left join Facilities as f on f.facid = b.facid
left join Members as m on m.memid = b.memid
where b.memid <> 0
group by facilityname
order by slot_usage desc;
"""
pd.read_sql_query(query, engine)



,facilityname,slot_usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


In [24]:
#Q13: Find the facilities usage by month, but not guests */

query = """
select sub1.month as month, sub1.facilityname as facility, sum(sub1.slotnumber) as slotusage
from
(select strftime('%m', starttime) as month, f.name as facilityname, b.slots as slotnumber
from Bookings b
left join Facilities f on f.facid = b.facid
left join Members m on m.memid = b.memid
where b.memid <> 0) as sub1
group by month, facility
order by month, slotusage desc;
"""
pd.read_sql_query(query, engine)




,month,facility,slotusage
0,07,Tennis Court 1,201
1,07,Massage Room 1,166
2,07,Badminton Court,165
3,07,Snooker Table,140
4,07,Tennis Court 2,123
5,07,Pool Table,110
6,07,Table Tennis,98
7,07,Squash Court,50
8,07,Massage Room 2,8
9,08,Badminton Court,414
